In [1]:
import joblib
import pandas as pd
import numpy as np
data = joblib.load("pcos_preprocessed.pkl")
data.head(2)

,Age,Weight_kg,Height_ft,Marital_Status,PCOS,Family_History_PCOS,Menstrual_Irregularity,Hormonal_Imbalance,Hyperandrogenism,Hirsutism,...,Diet_Multivitamin,Vegetarian,Exercise_Frequency,Exercise_Type,Stress_Level,Smoking,Exercise_Benefit,PCOS_Medication,Sleep_Hours_num,Exercise_Minutes
0,20-25,66.0,157.48,Unmarried,0,No,1,No,0,No,...,0,0,Rarely,"Cardio (e.g., running, cycling, swimming)",0,0,Somewhat,0.0,5.5,30
1,12-19,56.0,165.10,Unmarried,0,No,0,No,0,No,...,0,0,Daily,No Exercise,0,0,Somewhat,0.0,7.0,15


In [2]:
import joblib
pipeline = joblib.load("pipelines/pcos_preprocessing_pipeline.joblib")
preprocessor = pipeline.named_steps["preprocessor"]


In [3]:
from sklearn.model_selection import train_test_split

target_candidates = ["PCOS"]
X = data.drop(columns=target_candidates)
y = data["PCOS"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [4]:
print(y.value_counts())


PCOS
0    133
1     37
Name: count, dtype: int64


In [5]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [6]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = Pipeline([
    ("preprocessor", pipeline.named_steps["preprocessor"]),
    ("classifier", RandomForestClassifier(
        n_estimators=200,
        max_depth=10,
        class_weight="balanced",
        random_state=42
    ))
])
rf_clf.fit(X_train, y_train)
y_pred = rf_clf.predict(X_test)

print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      1.00      0.91        26
           1       1.00      0.38      0.55         8

    accuracy                           0.85        34
   macro avg       0.92      0.69      0.73        34
weighted avg       0.88      0.85      0.83        34

0.8529411764705882


In [7]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
smote.fit_resample(X_train, y_train)

ValueError: could not convert string to float: '20-25'

In [ ]:
# Convert to numeric
X_train_enc = preprocessor.transform(X_train)
X_test_enc = preprocessor.transform(X_test)


In [ ]:
X_res, y_res = smote.fit_resample(X_train, y_train)

# Use the preprocessor to transform categorical features
X_train_enc = pipeline.transform(X_train)
X_test_enc = pipeline.transform(X_test)

# Now apply SMOTE on the encoded numeric training data
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_train_enc, y_train)



ValueError: could not convert string to float: '20-25'

In [ ]:
print(classification_report(y_test, y_pred, target_names=["No PCOS","PCOS"]))
